In [30]:
from bisect      import bisect_left
from collections import deque, defaultdict
from functools   import reduce
from itertools   import combinations
from typing      import *

import math
import os
import pprint
import re

## Input functions

In [31]:
year = 2023

lines = str.splitlines
def paragraphs(text): return text.split('\n\n')

def parse(day_or_text: Union[str, int], parser: callable=str, splitter: callable=lines):
    """Splits the input file using `splitter` before parsing each split with `parser`"""
    text = get_text(day_or_text)
    records = mapt(parser, splitter(text.rstrip()))
    return records

def get_text(day_or_text: Union[str, int]):
    if isinstance(day_or_text, str):
        return day_or_text
    else:
        with open(f'../data/AOC/{year}/input{day_or_text}.txt') as f:
            return f.read()
    

In [40]:
Atom = Union[str, int, float]
Char = str

def ints(text: str) -> Tuple[int]:
    """A tuple of all the integers in text, ignoring non-number characters."""
    return mapt(int, re.findall(r'-?[0-9]+', text))

def pos_ints(text: str) -> Tuple[int]:
    """A tuple of all the positive ints in text."""
    return mapt(int, re.findall(r'\d+', text))

def digits(text: str) -> Tuple[int]:
    """A tuple of all the digits in text (as ints 0–9), ignoring non-digit characters."""
    return mapt(int, re.findall(r'[0-9]', text))

def chars(text: str) -> Tuple[Char]:
    """A tuple of all the chars in the text, including punctuation."""
    return mapt(Char, re.findall(r'\S', text))

def strings(text: str) -> Tuple[str]:
    return mapt(str, re.findall(r'\w+', text))

def words(text: str) -> Tuple[str]:
    return mapt(str, re.findall(r'[a-zA-Z]+', text))

def atoms(text: str) -> Tuple[Atom]:
    """A tuple of all the atoms (numbers or identifiers) in text. Skip punctuation."""
    return mapt(atom, re.findall(r'[+-]?\d+\.?\d*|\w+', text))

def atom(text: str) -> Atom:
    """Parse text into a single float or int or str."""
    try:
        x = float(text)
        return round(x) if x.is_integer() else x
    except ValueError:
        return text.strip()

Helper functions:

In [38]:
def mapt(function: Callable, *sequences) -> tuple:
    """`map`, with the result as a tuple."""
    return tuple(map(function, *sequences))

def mapl(function: Callable, *sequences) -> list:
    """`map`, with the result as a list."""
    return list(map(function, *sequences))

Tests for the functions above:

In [44]:
assert parse("hello\nworld") == ('hello', 'world')
assert parse("123\nabc7", digits) == ((1, 2, 3), (7,))

assert    atoms('hello, cruel_world! 24-7') == ('hello', 'cruel_world', 24, -7)
assert    words('hello, cruel_world! 24-7') == ('hello', 'cruel', 'world')
assert    chars('hello, ')                  == ('h', 'e', 'l', 'l', 'o' ,',')
assert   digits('hello, cruel_world! 24-7') == (2, 4, 7)
assert     ints('hello, cruel_world! 24-7') == (24, -7)
assert pos_ints('hello, cruel_world! 24-7') == (24, 7)

## Points in space

Cleaning up some code to make it easier to work with points that lie in some space. I'll have it so that each "point" takes in a variable number of integers that represents where its at in space

In [ ]:
Point = Tuple[int, ...]
Vector = Point                # E.g., (1, 0) can be a point, or can be a direction, a Vector
Zero = (0, 0)

directions4 = East, South, West, North = ((1, 0), (0, 1),  (-1, 0), (0, -1))
diagonals   = SE,   NE,    SW,   NW    = ((1, 1), (1, -1), (-1, 1), (-1, -1))
directions8 = directions4 + diagonals
directions5 = directions4 + (Zero,)
directions9 = directions8 + (Zero,)


def X_(point) -> int: "X coordinate of a point"; return point[0]
def Y_(point) -> int: "Y coordinate of a point"; return point[1]
def Z_(point) -> int: "Z coordinate of a point"; return point[2]

def Xs(points) -> Tuple[int]: "X coordinates of a collection of points"; return mapt(X_, points)
def Ys(points) -> Tuple[int]: "Y coordinates of a collection of points"; return mapt(Y_, points)
def Zs(points) -> Tuple[int]: "X coordinates of a collection of points"; return mapt(Z_, points)

def add2(p: Point, q: Point) -> Point: 
    """Specialized version of point addition for 2D Points only. Faster."""
    return (p[0] + q[0], p[1] + q[1])

def taxi_distance(p: Point, q: Point) -> int:
    """Manhattan (L1) distance between two 2D Points."""
    return abs(p[0] - q[0]) + abs(p[1] - q[1])

In [ ]:
def cells_around_8(x: int, y: int, include_center: bool=True):
    for xi in range(-1, 2):
        for yi in range(-1, 2):
            if not include_center and xi == 0 and yi == 0:
                continue
            yield (x + xi, y + yi)

def cells_around_4(x: int, y: int, include_center: bool=True):
    for xi in range(-1, 2):
        for yi in range(-1, 2):
            if not include_center and xi == 0 and yi == 0:
                continue
            if abs(xi) + abs(yi) > 1:
                continue
            yield (x + xi, y + yi)